In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_{}'.format('acgan')):
    os.mkdir('saved_models_{}'.format('acgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('acgan'), mode='w')

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import os
import numpy as np
import torch.utils.data as Data
class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test,y_test) = mnist.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)
        # Rescale -1 to 1
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                # Image labels. 0-9
                img_labels = y_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # The labels of the digits that the generator tries to create an
                # image representation of
                sampled_labels = np.random.randint(0, 10, (batch_size, 1))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, sampled_labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, [valid, img_labels])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss[0]))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
        


if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=30, batch_size=64, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 3.196865, acc.: 33.59%, op_acc: 10.16%] [G loss: 3.301222]
epoch:0 step:2 [D loss: 3.341656, acc.: 50.00%, op_acc: 8.59%] [G loss: 3.096192]
epoch:0 step:3 [D loss: 3.304219, acc.: 56.25%, op_acc: 13.28%] [G loss: 3.104835]
epoch:0 step:4 [D loss: 3.175223, acc.: 57.81%, op_acc: 15.62%] [G loss: 3.238871]
epoch:0 step:5 [D loss: 3.204794, acc.: 61.72%, op_acc: 9.38%] [G loss: 3.142260]
epoch:0 step:6 [D loss: 3.223628, acc.: 57.81%, op_acc: 12.50%] [G loss: 3.373064]
epoch:0 step:7 [D loss: 3.134950, acc.: 64.84%, op_acc: 9.38%] [G loss: 3.239519]
epoch:0 step:8 [D loss: 3.118153, acc.: 60.16%, op_acc: 11.72%] [G loss: 3.323327]
epoch:0 step:9 [D loss: 2.896620, acc.: 67.19%, op_acc: 16.41%] [G loss: 2.963837]
epoch:0 step:10 [D loss: 3.116403, acc.: 63.28%, op_acc: 9.38%] [G loss: 3.128019]
epoch:0 step:11 [D loss: 3.138301, acc.: 64.06%, op_acc: 15.62%] [G loss: 3.199135]
epoch:0 step:12 [D loss: 3.075962, acc.: 61.72%, op_acc: 13.28%] [G loss: 2.990057]
epoch

epoch:0 step:103 [D loss: 1.597006, acc.: 97.66%, op_acc: 50.00%] [G loss: 4.609185]
epoch:0 step:104 [D loss: 1.722650, acc.: 99.22%, op_acc: 50.00%] [G loss: 4.809923]
epoch:0 step:105 [D loss: 1.722008, acc.: 97.66%, op_acc: 42.19%] [G loss: 4.516505]
epoch:0 step:106 [D loss: 1.709765, acc.: 97.66%, op_acc: 46.88%] [G loss: 4.823711]
epoch:0 step:107 [D loss: 1.772052, acc.: 98.44%, op_acc: 48.44%] [G loss: 4.762057]
epoch:0 step:108 [D loss: 1.806568, acc.: 96.09%, op_acc: 43.75%] [G loss: 5.086296]
epoch:0 step:109 [D loss: 1.781686, acc.: 96.09%, op_acc: 40.62%] [G loss: 4.363954]
epoch:0 step:110 [D loss: 1.739456, acc.: 99.22%, op_acc: 43.75%] [G loss: 4.561930]
epoch:0 step:111 [D loss: 1.827243, acc.: 97.66%, op_acc: 46.09%] [G loss: 4.944924]
epoch:0 step:112 [D loss: 1.528399, acc.: 98.44%, op_acc: 53.91%] [G loss: 4.744928]
epoch:0 step:113 [D loss: 1.708342, acc.: 97.66%, op_acc: 48.44%] [G loss: 4.531961]
epoch:0 step:114 [D loss: 1.935891, acc.: 99.22%, op_acc: 36.72%]

...

epoch:29 step:27665 [D loss: 0.719413, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.723388]
epoch:29 step:27666 [D loss: 0.763324, acc.: 51.56%, op_acc: 98.44%] [G loss: 0.726206]
epoch:29 step:27667 [D loss: 0.766422, acc.: 52.34%, op_acc: 97.66%] [G loss: 0.742616]
epoch:29 step:27668 [D loss: 0.796810, acc.: 64.84%, op_acc: 97.66%] [G loss: 0.761337]
epoch:29 step:27669 [D loss: 0.721383, acc.: 56.25%, op_acc: 99.22%] [G loss: 0.789410]
epoch:29 step:27670 [D loss: 0.745777, acc.: 51.56%, op_acc: 98.44%] [G loss: 0.771188]
epoch:29 step:27671 [D loss: 0.738583, acc.: 50.78%, op_acc: 98.44%] [G loss: 0.713693]
epoch:29 step:27672 [D loss: 0.695097, acc.: 54.69%, op_acc: 98.44%] [G loss: 0.770254]
epoch:29 step:27673 [D loss: 0.705380, acc.: 57.81%, op_acc: 97.66%] [G loss: 0.753464]
epoch:29 step:27674 [D loss: 0.708481, acc.: 49.22%, op_acc: 99.22%] [G loss: 0.810269]
epoch:29 step:27675 [D loss: 0.738490, acc.: 50.78%, op_acc: 98.44%] [G loss: 0.770123]
epoch:29 step:27676 [D loss: 0.

epoch:29 step:27759 [D loss: 0.742255, acc.: 49.22%, op_acc: 98.44%] [G loss: 0.728577]
epoch:29 step:27760 [D loss: 0.718260, acc.: 47.66%, op_acc: 97.66%] [G loss: 0.714871]
epoch:29 step:27761 [D loss: 0.698675, acc.: 63.28%, op_acc: 99.22%] [G loss: 0.747311]
epoch:29 step:27762 [D loss: 0.708560, acc.: 57.03%, op_acc: 98.44%] [G loss: 0.748149]
epoch:29 step:27763 [D loss: 0.721573, acc.: 50.78%, op_acc: 99.22%] [G loss: 0.721534]
epoch:29 step:27764 [D loss: 0.772370, acc.: 46.09%, op_acc: 99.22%] [G loss: 0.714393]
epoch:29 step:27765 [D loss: 0.774787, acc.: 49.22%, op_acc: 98.44%] [G loss: 0.721640]
epoch:29 step:27766 [D loss: 0.717556, acc.: 51.56%, op_acc: 99.22%] [G loss: 0.723257]
epoch:29 step:27767 [D loss: 0.700153, acc.: 43.75%, op_acc: 100.00%] [G loss: 0.742863]
epoch:29 step:27768 [D loss: 0.717507, acc.: 52.34%, op_acc: 99.22%] [G loss: 0.746163]
epoch:29 step:27769 [D loss: 0.735729, acc.: 46.09%, op_acc: 99.22%] [G loss: 0.736993]
epoch:29 step:27770 [D loss: 0.

epoch:29 step:27855 [D loss: 0.759526, acc.: 50.78%, op_acc: 98.44%] [G loss: 0.750514]
epoch:29 step:27856 [D loss: 0.756589, acc.: 57.81%, op_acc: 98.44%] [G loss: 0.758548]
epoch:29 step:27857 [D loss: 0.693557, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.740534]
epoch:29 step:27858 [D loss: 0.747368, acc.: 46.09%, op_acc: 98.44%] [G loss: 0.731094]
epoch:29 step:27859 [D loss: 0.740819, acc.: 53.91%, op_acc: 97.66%] [G loss: 0.744247]
epoch:29 step:27860 [D loss: 0.728590, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.733881]
epoch:29 step:27861 [D loss: 0.733207, acc.: 56.25%, op_acc: 98.44%] [G loss: 0.731747]
epoch:29 step:27862 [D loss: 0.723780, acc.: 44.53%, op_acc: 98.44%] [G loss: 0.732846]
epoch:29 step:27863 [D loss: 0.687683, acc.: 56.25%, op_acc: 100.00%] [G loss: 0.737415]
epoch:29 step:27864 [D loss: 0.703095, acc.: 47.66%, op_acc: 100.00%] [G loss: 0.715686]
epoch:29 step:27865 [D loss: 0.702708, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.760902]
epoch:29 step:27866 [D loss:

epoch:29 step:27950 [D loss: 0.699028, acc.: 57.03%, op_acc: 99.22%] [G loss: 0.759543]
epoch:29 step:27951 [D loss: 0.719468, acc.: 52.34%, op_acc: 99.22%] [G loss: 0.755865]
epoch:29 step:27952 [D loss: 0.700327, acc.: 53.12%, op_acc: 100.00%] [G loss: 0.772981]
epoch:29 step:27953 [D loss: 0.863717, acc.: 50.78%, op_acc: 95.31%] [G loss: 0.750620]
epoch:29 step:27954 [D loss: 0.729795, acc.: 48.44%, op_acc: 99.22%] [G loss: 0.733087]
epoch:29 step:27955 [D loss: 0.737001, acc.: 47.66%, op_acc: 99.22%] [G loss: 0.723749]
epoch:29 step:27956 [D loss: 0.720736, acc.: 50.78%, op_acc: 99.22%] [G loss: 0.708244]
epoch:29 step:27957 [D loss: 0.707245, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.727864]
epoch:29 step:27958 [D loss: 0.778900, acc.: 50.00%, op_acc: 97.66%] [G loss: 0.705645]
epoch:29 step:27959 [D loss: 0.701995, acc.: 51.56%, op_acc: 99.22%] [G loss: 0.717179]
epoch:29 step:27960 [D loss: 0.806652, acc.: 53.12%, op_acc: 96.09%] [G loss: 0.756176]
epoch:29 step:27961 [D loss: 0

epoch:29 step:28045 [D loss: 0.732567, acc.: 49.22%, op_acc: 99.22%] [G loss: 0.693559]
epoch:29 step:28046 [D loss: 0.724581, acc.: 44.53%, op_acc: 100.00%] [G loss: 0.710502]
epoch:29 step:28047 [D loss: 0.735091, acc.: 46.88%, op_acc: 97.66%] [G loss: 0.719275]
epoch:29 step:28048 [D loss: 0.716632, acc.: 44.53%, op_acc: 100.00%] [G loss: 0.717763]
epoch:29 step:28049 [D loss: 0.701705, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.718020]
epoch:29 step:28050 [D loss: 0.716790, acc.: 46.09%, op_acc: 100.00%] [G loss: 0.726465]
epoch:29 step:28051 [D loss: 0.708818, acc.: 47.66%, op_acc: 100.00%] [G loss: 0.707013]
epoch:29 step:28052 [D loss: 0.724334, acc.: 52.34%, op_acc: 98.44%] [G loss: 0.729747]
epoch:29 step:28053 [D loss: 0.710427, acc.: 55.47%, op_acc: 99.22%] [G loss: 0.737624]
epoch:29 step:28054 [D loss: 0.686379, acc.: 57.81%, op_acc: 100.00%] [G loss: 0.725497]
epoch:29 step:28055 [D loss: 0.688300, acc.: 63.28%, op_acc: 99.22%] [G loss: 0.771377]
epoch:29 step:28056 [D los